<a href="https://colab.research.google.com/github/sahilk45/PyTorch-1Go/blob/main/Optuna_for_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [ ]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [ ]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2024-09-24 12:16:29,768] A new study created in memory with name: no-name-719abd5c-f02a-45f3-8f29-2fe3cbd51af6
[I 2024-09-24 12:16:30,753] Trial 0 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 179, 'max_depth': 14}. Best is trial 0 with value: 0.7765363128491621.
[I 2024-09-24 12:16:31,616] Trial 1 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 180, 'max_depth': 3}. Best is trial 0 with value: 0.7765363128491621.
[I 2024-09-24 12:16:32,046] Trial 2 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 80, 'max_depth': 5}. Best is trial 0 with value: 0.7765363128491621.
[I 2024-09-24 12:16:32,771] Trial 3 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 150, 'max_depth': 4}. Best is trial 0 with value: 0.7765363128491621.
[I 2024-09-24 12:16:33,436] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 119, 'max_depth': 14}. Best is trial 0 with value: 0.77653631

In [ ]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 113, 'max_depth': 18}


In [ ]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## Samplers in Optuna

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2024-09-24 11:15:07,436] A new study created in memory with name: no-name-94a921ff-de95-4ecf-9bbe-82aeb1eadeb5
[I 2024-09-24 11:15:08,234] Trial 0 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 146, 'max_depth': 20}. Best is trial 0 with value: 0.7709497206703911.
[I 2024-09-24 11:15:09,260] Trial 1 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 195, 'max_depth': 9}. Best is trial 0 with value: 0.7709497206703911.
[I 2024-09-24 11:15:09,683] Trial 2 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 75, 'max_depth': 12}. Best is trial 0 with value: 0.7709497206703911.
[I 2024-09-24 11:15:10,382] Trial 3 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 139, 'max_depth': 5}. Best is trial 0 with value: 0.7709497206703911.
[I 2024-09-24 11:15:11,315] Trial 4 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 172, 'max_depth': 15}. Best is trial 4 with value: 0.7746741

In [ ]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 115, 'max_depth': 18}


In [ ]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [ ]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [ ]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2024-09-24 11:16:07,889] A new study created in memory with name: no-name-85e601bb-fd83-4bc3-bba3-042b9d6dbde9
[I 2024-09-24 11:16:09,697] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-09-24 11:16:10,647] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-09-24 11:16:10,937] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-09-24 11:16:11,481] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-09-24 11:16:12,030] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [ ]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [ ]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [ ]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [ ]:
# 1. Optimization History
plot_optimization_history(study).show()

In [ ]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [ ]:
# 3. Slice Plot
plot_slice(study).show()

In [ ]:
# 4. Contour Plot
plot_contour(study).show()

In [ ]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [ ]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [ ]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-09-21 19:20:25,685] A new study created in memory with name: no-name-0cb136d9-bc09-4c7e-bfe7-428ef9952939
[I 2024-09-21 19:20:26,106] Trial 0 finished with value: 0.7858472998137801 and parameters: {'classifier': 'SVM', 'C': 49.91032649314737, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 0 with value: 0.7858472998137801.
[I 2024-09-21 19:20:26,162] Trial 1 finished with value: 0.7597765363128491 and parameters: {'classifier': 'SVM', 'C': 0.26095994150012275, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.7858472998137801.
[I 2024-09-21 19:20:26,217] Trial 2 finished with value: 0.7262569832402234 and parameters: {'classifier': 'SVM', 'C': 0.10964808239396216, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.7858472998137801.
[I 2024-09-21 19:20:28,165] Trial 3 finished with value: 0.7653631284916201 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 197, 'learning_rate': 0.01865676538441911, 'max_depth': 5, 'min_samples_

In [ ]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.12914668553776398, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.785847,2024-09-21 19:20:25.690456,2024-09-21 19:20:26.105462,0 days 00:00:00.415006,49.910326,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.759777,2024-09-21 19:20:26.111879,2024-09-21 19:20:26.162152,0 days 00:00:00.050273,0.260960,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.726257,2024-09-21 19:20:26.165386,2024-09-21 19:20:26.217181,0 days 00:00:00.051795,0.109648,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.765363,2024-09-21 19:20:26.220264,2024-09-21 19:20:28.164721,0 days 00:00:01.944457,NaN,NaN,GradientBoosting,NaN,NaN,0.018657,5.0,3.0,4.0,197.0,COMPLETE
4,4,0.783985,2024-09-21 19:20:28.167941,2024-09-21 19:20:28.213761,0 days 00:00:00.045820,1.071288,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.789572,2024-09-21 19:21:10.060861,2024-09-21 19:21:10.092814,0 days 00:00:00.031953,0.117532,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.787709,2024-09-21 19:21:10.094623,2024-09-21 19:21:10.126118,0 days 00:00:00.031495,0.101268,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.770950,2024-09-21 19:21:10.127850,2024-09-21 19:21:10.165977,0 days 00:00:00.038127,1.547787,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.787709,2024-09-21 19:21:10.167751,2024-09-21 19:21:10.201000,0 days 00:00:00.033249,0.168769,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [ ]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,80
GradientBoosting,10
RandomForest,10


In [ ]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.739851
RandomForest,0.763873
SVM,0.774977


In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2024-09-21 20:35:31,885] A new study created in memory with name: no-name-949dd358-8192-4acd-b18a-84e91902369d


[0]	train-mlogloss:0.86599	eval-mlogloss:0.85820
[1]	train-mlogloss:0.69848	eval-mlogloss:0.69525
[2]	train-mlogloss:0.57389	eval-mlogloss:0.56735
[3]	train-mlogloss:0.48215	eval-mlogloss:0.47291
[4]	train-mlogloss:0.40412	eval-mlogloss:0.38894
[5]	train-mlogloss:0.34451	eval-mlogloss:0.32079
[6]	train-mlogloss:0.29662	eval-mlogloss:0.27008
[7]	train-mlogloss:0.25758	eval-mlogloss:0.22937
[8]	train-mlogloss:0.22504	eval-mlogloss:0.19268
[9]	train-mlogloss:0.19847	eval-mlogloss:0.16039
[10]	train-mlogloss:0.17634	eval-mlogloss:0.13676
[11]	train-mlogloss:0.15773	eval-mlogloss:0.11718
[12]	train-mlogloss:0.14373	eval-mlogloss:0.09947
[13]	train-mlogloss:0.12877	eval-mlogloss:0.08456
[14]	train-mlogloss:0.11568	eval-mlogloss:0.07359
[15]	train-mlogloss:0.10477	eval-mlogloss:0.06427
[16]	train-mlogloss:0.09706	eval-mlogloss:0.05765
[17]	train-mlogloss:0.08948	eval-mlogloss:0.05320
[18]	train-mlogloss:0.08518	eval-mlogloss:0.04887
[19]	train-mlogloss:0.08224	eval-mlogloss:0.04516
[20]	train

[I 2024-09-21 20:35:35,212] Trial 0 finished with value: 1.0 and parameters: {'lambda': 5.0537889201945e-07, 'alpha': 0.006355225822262458, 'eta': 0.18175383524617775, 'gamma': 0.5586379771196535, 'max_depth': 6, 'min_child_weight': 1, 'subsample': 0.701721665674433, 'colsample_bytree': 0.8304727267992562}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:1.02436	eval-mlogloss:1.02721
[1]	train-mlogloss:0.95829	eval-mlogloss:0.96340
[2]	train-mlogloss:0.89182	eval-mlogloss:0.89332
[3]	train-mlogloss:0.84077	eval-mlogloss:0.83926
[4]	train-mlogloss:0.78582	eval-mlogloss:0.78025
[5]	train-mlogloss:0.74341	eval-mlogloss:0.73625
[6]	train-mlogloss:0.69744	eval-mlogloss:0.68839
[7]	train-mlogloss:0.65536	eval-mlogloss:0.64235
[8]	train-mlogloss:0.62275	eval-mlogloss:0.60956
[9]	train-mlogloss:0.59304	eval-mlogloss:0.57895
[10]	train-mlogloss:0.55947	eval-mlogloss:0.54325
[11]	train-mlogloss:0.52812	eval-mlogloss:0.50907
[12]	train-mlogloss:0.49909	eval-mlogloss:0.47772
[13]	train-mlogloss:0.47251	eval-mlogloss:0.44937
[14]	train-mlogloss:0.44757	eval-mlogloss:0.42235
[15]	train-mlogloss:0.42683	eval-mlogloss:0.40116
[16]	train-mlogloss:0.40731	eval-mlogloss:0.38167
[17]	train-mlogloss:0.38685	eval-mlogloss:0.35995
[18]	train-mlogloss:0.36773	eval-mlogloss:0.33991
[19]	train-mlogloss:0.35009	eval-mlogloss:0.32079
[20]	train

[I 2024-09-21 20:35:39,002] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.0005923786887942196, 'alpha': 0.007944527162657351, 'eta': 0.062325575391692846, 'gamma': 2.2239369972976245e-07, 'max_depth': 5, 'min_child_weight': 6, 'subsample': 0.9581180380017678, 'colsample_bytree': 0.5234298261097905}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.03911	eval-mlogloss:1.03759


[I 2024-09-21 20:35:50,986] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:1.05017	eval-mlogloss:1.04882


[I 2024-09-21 20:35:51,053] Trial 49 pruned. Trial was pruned at iteration 1.


Best trial: {'lambda': 5.0537889201945e-07, 'alpha': 0.006355225822262458, 'eta': 0.18175383524617775, 'gamma': 0.5586379771196535, 'max_depth': 6, 'min_child_weight': 1, 'subsample': 0.701721665674433, 'colsample_bytree': 0.8304727267992562}
Best accuracy: 1.0


In [ ]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 3.1 MB/s eta 0:00:00
